# This notebook formats the data, and runs it through our bidirectional transformer architecture.

### Data formatting:

Refer to this notebook: https://github.com/ocatak/lstm_malware_detection/blob/master/deep_learnin_lstm_malware_detection.ipynb

### Bidirectional architecture:

... to be added

In [16]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, './keras_bert_rec')
import importlib
import keras_bert_rec as rec
importlib.reload(rec)

import argparse
import keras
import numpy as np
import pandas as pd
import pickle
import sys
import tensorflow as tf
import importlib

from itertools import chain
from keras import backend as K
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from sklearn.preprocessing import OneHotEncoder

In [10]:
! pip install keras_bert

  Stored in directory: /home/serena/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
Successfully built keras-bert
  Found existing installation: Keras 2.2.3
    Uninstalling Keras-2.2.3:
      Successfully uninstalled Keras-2.2.3
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
DATA_PATH = "../data/"

In [ ]:
# Read in data
## Calls
malware_calls_df = pd.read_csv(f"{DATA_PATH}calls.zip", compression="zip",
                               sep="\t", names=["API_Calls"])


In [ ]:
## Labels ("types")
malware_labels_df = pd.read_csv(f"{DATA_PATH}types.zip", compression="zip",
                               sep="\t", names=["API_Labels"])


In [ ]:
## Concat data
malware_calls_df["API_Labels"] = malware_labels_df.API_Labels
malware_calls_df["API_Calls"] = malware_calls_df.API_Calls.apply(lambda x: " ".join(x.split(",")))

malware_calls_df["API_Labels"] = malware_calls_df.API_Labels.apply(lambda x: 1 if x == "Virus" else 0)

In [ ]:
max_words = 800
max_len = 100

X = malware_calls_df.API_Calls
Y = malware_calls_df.API_Labels.astype('category').cat.codes

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SpatialDropout1D
# from mlxtend.plotting import plot_confusion_matrix

In [ ]:
malware_calls_df

In [ ]:
X

In [ ]:
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
print('Found %s unique tokens.' % len(tok.word_index))
X = tok.texts_to_sequences(X.values)
X = sequence.pad_sequences(X, maxlen=max_len)
print('Shape of data tensor:', X.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.15)

le = LabelEncoder()
Y_train_enc = le.fit_transform(Y_train)
Y_train_enc = np_utils.to_categorical(Y_train_enc)

Y_test_enc = le.transform(Y_test)
Y_test_enc = np_utils.to_categorical(Y_test_enc)